In [1]:
%%time

# Trace evaluation domain and FRI evaluation domain
# G = trace evaluation domain
# L = FRI evaluation domain
# G and L are disjoint
# |G| < |L|
# |G| is a power of 2 (to use FFT)
# |L| is a power of 2 (to use FFT)
# F[P] = Prime field of order P, P is prime
# F[P, *] = Multiplicative subgroup of F[P] = {1, 2, 3, ..., P - 1}
# |F[P, *]| = P - 1
# G and L are subgroups of F[P, *] -> |G| and |L| divides |F[P, *]| = P - 1

import fft_poly
import iop
import merkle
import numpy as np
import polynomial
import stark
from fft import eval_poly, fft, ifft
from field import F, find_generator, generate, get_primitive_root
from polynomial import Polynomial, X

from utils import is_pow2, is_prime, min_pow2_gt

### Example claim = trace is all 0 or 1 ###
T = 1024
trace = [int(i) for i in np.random.choice([0, 1], size=T)]

### Setup for evaluation domains - pick prime field and a generator ###
# Prime field F[P]
# Starkware prime
# P = 2**251 + 17 * 2**192 + 1
P = 3 * 2**30 + 1
assert is_prime(P)

# Generator of F[P, *] - multiplicative group of the prime field F[P]
# g must be known by the prover and the verifier
g = find_generator(P)
print("Generator of F[P, *] = g =", g)

### Trace evaluation domain ###
# Let G = trace evaluation domain
# Find G such that |G| = T
# T must be a power of 2 to use FFT
# |G| must divide P - 1 so that |G| is a subgroup of F[P, *]
assert is_pow2(T)
assert (P - 1) % T == 0
trace_eval_domain = generate(get_primitive_root(g, T, P), T, P)

### Trace polynomial ###
# f(x) = trace polynomial
# f(G[i]) = trace[i]
f = fft_poly.interp(trace, trace_eval_domain, P)
assert fft_poly.eval(f, trace_eval_domain, P) == trace, "f(G) != trace"
print("Degree of trace polynomial =", f.degree())

### Constraint polynomial ###
# c(x) = constraint polynomial
# trace[i] is either 0 or 1 -> trace[i](trace[i] - 1) = 0 for all G[i]
#                           -> f(x)(f(x) - 1) = 0 for all G[i]
#                           -> Set c(x) = f(x)(f(x) - 1)
c = f * f - f
print("Degree of constraint polynomial =", c.degree())

EXP_FACTOR = 8
N = EXP_FACTOR * T

prover = stark.Prover(
    P=P,
    g=g,
    trace_poly=f,
    trace_eval_domain=trace_eval_domain,
    exp_factor=EXP_FACTOR,
    constraint_poly=c
)

x = X(1, lambda x: F(x, P))

verifier = stark.Verifier(
    P=P,
    g=g,
    trace_len=T,
    exp_factor=EXP_FACTOR,
    constraint_poly=x * x - x
)

### Communication channels ###
chan = iop.Channel(iop.Prover(prover), iop.Verifier(verifier))

### STARK and FRI - prover commit ###
prover.commit(chan)

### STARK - verifier query ###
# Preliminary check before queries
verifier.check()

NUM_QUERIES = 50
assert NUM_QUERIES <= N

Q = [int(i) for i in np.random.choice(range(N), size=NUM_QUERIES, replace=False)]
for i in Q:
    verifier.query(i, chan)

### FRI - verifier query ###
for i in Q:
    verifier.fri().query(i, chan)

print("All checks passed ✅️")

# TODO - attacks
# TODO - probabilities?
# TODO: pair f(x) and f(-x) in merkle tree?

Generator of F[P, *] = g = 5
Degree of trace polynomial = 1023
Degree of constraint polynomial = 2046
All checks passed ✅️
CPU times: user 2.1 s, sys: 4.6 ms, total: 2.11 s
Wall time: 1.78 s


In [1]:
# A Walk-Through of a Simple zk-STARK Proof

# N = 4
# A = [1, 0, 1, 1]
# g = 4 mod 17
# f(g[i]) = A[i] for i < N
# A[i] = 0 or 1 -> A[i](A[i] - 1) = 0 for i < N -> f(g[i])^2 - f(g[i]) = 0
# c(x) = f(x)^2 - f(x)
# u(x) = ((x-g[0])(x - g[1])...(x - g[N - 1])) = x^N - 1
# p(x) = c(x) / u(x)
# p(x) is polynomial of degree <= deg(c) - N <-> c(x) = 0 for x = g[i] <-> All A[i] = 0 or 1

# Query
# Verifier checks p(x)u(x) = c(x) (query x in L - G, query x in G breaks zero knowledge)
# 1. Verifier queries for z in L - G
# 2. Prover sends p(z) and f(z) and Merkle proofs of p(z) and f(z)
# 3. Verifier checks p(z)u(z) = c(z) and Merkle proofs

# num queries for high probability of fraud detection
# poly degree = d = 5
# evaluation domain L = 100
# num of dishonest evaluation = |L| - d = 95
# probability of getting caught after k queries = 1 - probability of not getting caught after k queries ~ 1 - (d / |L|)^k

# TODO - cheat if p is not a low degree
# TODO - example of false trace?

# Apply FRI to p

Generator of F[P, *] = 5
Generator of G = 1855261384
Generator of L = 1734477367


In [3]:
set() == {}

False